In [1]:
import HUXt as H
import HIEnsembleHindcast as heh
import HI_analysis as hip
import tables
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import moviepy.editor as mpy
from moviepy.video.io.bindings import mplfig_to_npimage
import numpy as np
import pandas as pd
import os 
from astropy.time import Time, TimeDelta
import scipy.stats as st
import glob

In [ ]:
event_list = ['ssw_007' 'ssw_008', 'ssw_009', 'ssw_012']
for event in event_list:
    print('***************************')
    print(event)   
    heh.run_huxt_ensemble(event, n_ensemble=200)
    heh.track_cme_flanks(event, fast=True)    
    heh.compute_ssw_profile(event)
    heh.compute_ensemble_metrics(event)
    time = 0.4*u.day
    heh.plot_huxt_and_hi_schematic(event, time)
    heh.plot_ensemble_elongation_profiles(event)
    heh.print_stats_table(event)
    heh.ensemble_histogram(event)
    heh.scatter_weight_error(event)
    
heh.all_time_elongation_plots()
heh.all_weight_vs_error_plots()
heh.print_summary_statistics()